In [17]:
import pandas as pd
import numpy as np
import glob
import gc
import os
import math
import keras
from keras.layers import CuDNNLSTM, Dense, TimeDistributed, Dropout, MaxPooling2D, Conv2D, Input
from keras.layers import Flatten, BatchNormalization, Activation, Reshape, concatenate
from keras.callbacks import LambdaCallback
from keras.models import Model
from keras import regularizers
import random
from sklearn.model_selection import KFold
import keras.backend as K
from sklearn.utils import class_weight
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm

from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.xception import Xception
from keras.applications.densenet import DenseNet201
from keras.applications.nasnet import NASNetLarge
from keras.applications.mobilenet_v2 import MobileNetV2

gc.enable()

In [18]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [19]:
from modules import clr, LRFinder
from modules.utils import DataGenerator

In [20]:
import tensorflow as tf
import random

def seed_everything(seed=2019):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                                  inter_op_parallelism_threads=1)
    tf.set_random_seed(seed)

    sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
    K.set_session(sess)
    
seed_everything()


In [21]:
train = pd.read_csv('data/train.csv')

# load data

In [6]:
print(train.head())

    itemid                                              title  Category  \
0   307504               nyx sex bomb pallete natural palette         0   
1   461203  etude house precious mineral any cushion pearl...         1   
2  3592295                           milani rose powder blush         2   
3  4460167                etude house baby sweet sugar powder         3   
4  5853995       bedak revlon color stay aqua mineral make up         3   

                                          image_path  
0  beauty_image/6b2e9cbb279ac95703348368aa65da09.jpg  
1  beauty_image/20450222d857c9571ba8fa23bdedc8c9.jpg  
2  beauty_image/6a5962bed605a3dd6604ca3a4278a4f9.jpg  
3  beauty_image/56987ae186e8a8e71fcc5a261ca485da.jpg  
4  beauty_image/9c6968066ebab57588c2f757a240d8b9.jpg  


In [7]:
beauty_class = np.arange(0, 17)

In [8]:
batchsize = 32
height = 299
width = 299
seed = 2019
path = ""

In [9]:
indexes = np.load('np_array/trn_index_b_fold.npy')
val_index = indexes[0]
trn_index = indexes[1]
for i in range(2,4):
    trn_index = np.append(trn_index, indexes[i])

In [10]:
def simple_FC_layer(cv_input, classes):
    cv_input_dense = Flatten()(cv_input.output)
    main_ = Dropout(0.5)(cv_input_dense)
    main_ = Dense(300)(main_)
    output = Dense(len(classes), activation='softmax')(main_)
    model = Model(inputs=cv_input.input, outputs=output)
    return model

In [11]:
img_paths_trn = list(train.iloc[trn_index, 3])
labels_trn = train.iloc[trn_index, 2]
labels_trn = np.array(labels_trn) - beauty_class[0]
labels_trn = keras.utils.to_categorical(labels_trn, num_classes=len(beauty_class))

img_paths_val = list(train.iloc[val_index, 3])
labels_val = train.iloc[val_index, 2]
labels_val = np.array(labels_val) - beauty_class[0]
labels_val = keras.utils.to_categorical(labels_val, num_classes=len(beauty_class))

params = {'dim': (299, 299),
          'batch_size': 32,
          'n_channels': 3,
          'shuffle': True,
          "path": path
         }

training_generator = DataGenerator(img_paths_trn, labels_trn, beauty_class, **params)
testing_generator = DataGenerator(img_paths_val, labels_val, beauty_class, **params)

In [12]:
input_tensor=Input(shape=(height, width, 3))
backbone_model = InceptionResNetV2(weights='imagenet', include_top=False, input_tensor=input_tensor)
model = simple_FC_layer(backbone_model, beauty_class)

In [13]:
import imgaug as ia
from imgaug import augmenters as iaa
import numpy as np

ia.seed(1)

# Example batch of images.
# The array has shape (32, 64, 64, 3) and dtype uint8.
images = np.array(
    [ia.quokka(size=(64, 64)) for _ in range(32)],
    dtype=np.uint8
)

seq = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontal flips
    iaa.Crop(percent=(0, 0.1)), # random crops
    # Small gaussian blur with random sigma between 0 and 0.5.
    # But we only blur about 50% of all images.
    iaa.Sometimes(0.5,
        iaa.GaussianBlur(sigma=(0, 0.5))
    ),
    # Strengthen or weaken the contrast in each image.
    iaa.ContrastNormalization((0.75, 1.5)),
    # Add gaussian noise.
    # For 50% of all images, we sample the noise once per pixel.
    # For the other 50% of all images, we sample the noise per pixel AND
    # channel. This can change the color (not only brightness) of the
    # pixels.
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
    # Make some images brighter and some darker.
    # In 20% of all cases, we sample the multiplier once per channel,
    # which can end up changing the color of the images.
    iaa.Multiply((0.8, 1.2), per_channel=0.2),
    # Apply affine transformations to each image.
    # Scale/zoom them, translate/move them, rotate them and shear them.
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-25, 25),
        shear=(-8, 8)
    )
], random_order=True) # apply augmenters in random order

In [14]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, image_paths, labels, classes, batch_size=128, dim=(299, 299), n_channels=3,
                 shuffle=True, path=None):
        'Initialization'
        self.path = path
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.image_paths = image_paths
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.classes = classes
        self.n_classes = len(classes)
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.image_paths))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size, self.n_classes), dtype=int)

        # Generate data
        for i, idx in enumerate(indexes):
            # Store sample
            # X[i,] = np.load('data/' + ID + '.npy')
            path_ = self.image_paths[idx]
            image = cv2.imread(self.path+path_)
            if image.size != (299, 299, 3):
                image = cv2.resize(image, (299, 299))
            image = seq.augment_image(image)
            X[i,] = image/255

            # Store class
            y[i] = self.labels[idx]
        
        return X, y
training_generator_aug = DataGenerator(img_paths_trn, labels_trn, beauty_class, **params)

In [15]:
cyclic_lr = clr.CyclicLR(base_lr=5e-5, max_lr=1e-4, step_size=3500)
adam = keras.optimizers.adam(decay=1e-5)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(generator = training_generator_aug, 
                        validation_data = testing_generator,
                        epochs = 7,
                        verbose = 1,
                        callbacks=[cyclic_lr],
                        workers = 4,
                        use_multiprocessing=True
                        )

Epoch 1/7
5373/5373 [==============================] - 3106s 578ms/step - loss: 1.4884 - acc: 0.5479 - val_loss: 1.1835 - val_acc: 0.6364
Epoch 2/7
5373/5373 [==============================] - 3043s 566ms/step - loss: 1.1941 - acc: 0.6374 - val_loss: 1.5541 - val_acc: 0.6096
Epoch 3/7
5373/5373 [==============================] - 3063s 570ms/step - loss: 1.1196 - acc: 0.6583 - val_loss: 1.0864 - val_acc: 0.6691
Epoch 4/7
5373/5373 [==============================] - 3022s 562ms/step - loss: 1.0569 - acc: 0.6775 - val_loss: 1.0413 - val_acc: 0.6844
Epoch 5/7
5373/5373 [==============================] - 2995s 557ms/step - loss: 1.0387 - acc: 0.6847 - val_loss: 1.0454 - val_acc: 0.6814
Epoch 6/7
5373/5373 [==============================] - 3050s 568ms/step - loss: 0.9815 - acc: 0.6996 - val_loss: 1.0495 - val_acc: 0.6820
Epoch 7/7
5373/5373 [==============================] - 3046s 567ms/step - loss: 0.9494 - acc: 0.7079 - val_loss: 1.0217 - val_acc: 0.6910


KeyboardInterrupt: 

In [24]:
model.load_weights('weights/beauty_0')

In [25]:
class TestGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, image_paths, batch_size=1, dim=(299, 299), n_channels=3,
                 shuffle=True, path=''):
        'Initialization'
        self.path = path
        self.dim = dim
        self.batch_size = batch_size
        self.image_paths = image_paths
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X = self.__data_generation(indexes)

        return X

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.image_paths))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))

        # Generate data
        for i, idx in enumerate(indexes):
            # Store sample
            # X[i,] = np.load('data/' + ID + '.npy')
            path_ = self.image_paths[idx]
            image = cv2.imread(self.path+path_)
            if image.size != (299, 299, 3):
                image = cv2.resize(image, (299, 299))
            X[i,] = image/255
        
        return X

In [26]:
output = model.predict_generator(testing_generator, verbose=1)
np.save('cv_beauty_0.npy', output)
test = pd.read_csv('data/test.csv')
img_paths = np.array(test.iloc[:76545, 2])
test_generator = TestGenerator(img_paths, shuffle=False, batch_size=64)
output = model.predict_generator(test_generator, verbose=True)
new_img_paths = img_paths[img_paths.shape[0]-img_paths.shape[0]%64:, ]
new_test_generator = TestGenerator(new_img_paths, shuffle=False, batch_size=1)
output_temp = model.predict_generator(new_test_generator, verbose=True)
output = np.append(output, output_temp, axis=0)
np.save('cv_beauty_0_test.npy', output)
model_embed = Model(inputs = model.input, outputs=model.layers[-2].output)
output = model_embed.predict_generator(testing_generator, verbose=True)
np.save('cv_beauty_0_embed.npy', output)
output = model_embed.predict_generator(test_generator, verbose=True)
output_temp = model_embed.predict_generator(new_test_generator, verbose=True)
output = np.append(output, output_temp, axis=0)
np.save('cv_beauty_0_test_embed.npy', output)

1/1 [==============================] - 0s 41ms/step
